# Lab 7: Performance - Splitting high cardinality columns

**Split Columns**
- Cold cache performance
- Model Size
- Warm cache performance


## 1. Install Semantic Link Labs Python Library

In [3]:
%pip install -q semantic-link-labs

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 9, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



## 2. Install Python Libraries

In [4]:
import sempy_labs as labs
from sempy import fabric
import sempy
import pandas
import time
import json

LakehouseName = "Performance_Lakehouse"
SemanticModelName = f"{LakehouseName}_model"
Shortcut_LakehouseName = "High_Cardinality_Demo_Lakehouse"
Shortcut_WorkspaceName = "Big Data DB Demo"

def myDeltaAnalyzer(table_name:str):
    # Run Delta Analyzer
    analyzer:dict = labs.delta_analyzer(lakehouse=LakehouseName, table_name=table_name,skip_cardinality=True)
    for key , value in analyzer.items():
        displayHTML(f"<H2>#### {key} ({table_name}) ####</H2>")
        display(value)

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 11, Finished, Available, Finished)

## 3. Create Lakehouse

In [5]:
lakehouses=labs.list_lakehouses()["Lakehouse Name"]
if LakehouseName in lakehouses.values:
    lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]
else:
    lakehouseId = fabric.create_lakehouse(LakehouseName)

workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
workspaceName = sempy.fabric.resolve_workspace_name(workspaceId)
print(f"WorkspaceId = {workspaceId}, LakehouseID = {lakehouseId}, Workspace Name = {workspaceName}")

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 12, Finished, Available, Finished)

WorkspaceId = 2435ad0e-cbcd-4737-8697-7bf7972c0360, LakehouseID = 570d1511-e48a-4476-855e-1cdde66074a1, Workspace Name = Comprehensive Guide to Direct Lake - Workshop Assets


## 4. Create Lakehouse Shortcuts

In [7]:
#1. Remove any existing shortcuts
for index, row in labs.lakehouse.list_shortcuts(lakehouse=LakehouseName).iterrows():
    labs.lakehouse.delete_shortcut(shortcut_name=row["Shortcut Name"],lakehouse=LakehouseName)
    print(f"Deleted shortcut {row['Shortcut Name']}")

time.sleep(3)

#2. Creates correct shortcuts
labs.lakehouse.create_shortcut_onelake(table_name="Dim_Attributes"          ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Dim_CustomerAttributes"  ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Dim_Date"                ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Dim_DeviceAttributes"    ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Fact_A_Level"            ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Fact_A_Level_Split_1"    ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)
labs.lakehouse.create_shortcut_onelake(table_name="Fact_A_Level_Split_2"    ,source_lakehouse=Shortcut_LakehouseName,source_workspace=Shortcut_WorkspaceName,destination_lakehouse=LakehouseName)

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 14, Finished, Available, Finished)

🟢 The shortcut 'Dim_Attributes' was created in the 'Performance_Lakehouse' lakehouse within the 'Comprehensive Guide to Direct Lake - Workshop Assets' workspace. It is based on the 'Dim_Attributes' table in the 'High_Cardinality_Demo_Lakehouse' lakehouse within the 'Big Data DB Demo' workspace.
🟢 The shortcut 'Dim_CustomerAttributes' was created in the 'Performance_Lakehouse' lakehouse within the 'Comprehensive Guide to Direct Lake - Workshop Assets' workspace. It is based on the 'Dim_CustomerAttributes' table in the 'High_Cardinality_Demo_Lakehouse' lakehouse within the 'Big Data DB Demo' workspace.
🟢 The shortcut 'Dim_Date' was created in the 'Performance_Lakehouse' lakehouse within the 'Comprehensive Guide to Direct Lake - Workshop Assets' workspace. It is based on the 'Dim_Date' table in the 'High_Cardinality_Demo_Lakehouse' lakehouse within the 'Big Data DB Demo' workspace.
🟢 The shortcut 'Dim_DeviceAttributes' was created in the 'Performance_Lakehouse' lakehouse within the 'Compr

## 5. Trigger background job to sync Lakehouse tables

In [8]:
##https://medium.com/@sqltidy/delays-in-the-automatically-generated-schema-in-the-sql-analytics-endpoint-of-the-lakehouse-b01c7633035d

def triggerMetadataRefresh():
    client = fabric.FabricRestClient()
    response = client.get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouseId}")
    sqlendpoint = response.json()['properties']['sqlEndpointProperties']['id']

    # trigger sync
    uri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}"
    payload = {"commands":[{"$type":"MetadataRefreshExternalCommand"}]}
    response = client.post(uri,json= payload)
    batchId = response.json()['batchId']

    # Monitor Progress
    statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
    statusresponsedata = client.get(statusuri).json()
    progressState = statusresponsedata['progressState']
    print(progressState)
    while progressState != "success":
        statusuri = f"/v1.0/myorg/lhdatamarts/{sqlendpoint}/batches/{batchId}"
        statusresponsedata = client.get(statusuri).json()
        progressState = statusresponsedata['progressState']
        print(progressState)
        time.sleep(1)

    print('done')

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 15, Finished, Available, Finished)

## Show column data

In [34]:
path = f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/Fact_A_Level_Split_1"
delta_table = spark.read.load(path)

display(delta_table.limit(20).orderBy(col("Metric").desc()))

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6d2a9db8-dace-4c14-8b26-5fa1cdf784a5)

## How many unique values exist in the Metric column

In [22]:
# how many unique values in Metric column?  

print(delta_table.select("Metric").distinct().count())              #   44,257,013

print(delta_table.select("Metric_x0").distinct().count())           #       10,001     
print(delta_table.select("Metric_x10000").distinct().count())       #       10,001
print(delta_table.select("Metric_x100000000").distinct().count())   #        1,904


StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 29, Finished, Available, Finished)

44257013


## 6. Run Delta Analyzer to look at size of Metric column in Lakehouse Delta Tables

The first block of code runs the full Delta Analyzer over three tables.  

Once the results are stored in a variable, we can run queries over the results without needing to run Delta Analyzer again

- **Fact_A_Level**          This is the base table with a high cardinally column
- **Fact_A_Level_Split_1**   Contains the original Metric column, plus the new split cols
- **Fact_A_Level_Split_2**   Contains only the new split cols

In [35]:
df1:dict[str, pandas.DataFrame] = labs.delta_analyzer(lakehouse=LakehouseName, workspace=workspaceName, table_name="Fact_A_Level"          ,skip_cardinality=True)
df2:dict[str, pandas.DataFrame] = labs.delta_analyzer(lakehouse=LakehouseName, workspace=workspaceName, table_name="Fact_A_Level_Split_1"  ,skip_cardinality=True)
df3:dict[str, pandas.DataFrame] = labs.delta_analyzer(lakehouse=LakehouseName, workspace=workspaceName, table_name="Fact_A_Level_Split_2"  ,skip_cardinality=True)

print("Done")

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 549e7e15-ae31-4e1d-99ce-87e886948e28)

SynapseWidget(Synapse.DataFrame, adb062f9-9752-466a-9476-b65c7d158be9)

🟢 Mounted the 'Performance_Lakehouse' lakehouse within the 'Comprehensive Guide to Direct Lake - Workshop Assets' to the notebook.


SynapseWidget(Synapse.DataFrame, f95d73ea-56df-449b-84c5-8752a28c1578)

...

SynapseWidget(Synapse.DataFrame, 05af4395-d5d1-43cf-bfa4-a9dd68cd2114)

SynapseWidget(Synapse.DataFrame, ab4767f3-1bea-4633-80f2-20ab55f45dfd)

  0%|          | 0/30 [00:00<?, ?it/s]

Done


StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 44, Finished, Available, Finished)

### Review Delta Analyzer results

In [36]:
for key , value in df1.items():
    print(key)

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 43, Finished, Available, Finished)

Summary
Parquet Files
Row Groups
Column Chunks
Columns


Review Summary statistics for all three Delta Tables

In [38]:
# display(df1["Summary"])
# display(df2["Summary"])
# display(df3["Summary"])

display(
    pandas.concat([
        df1["Summary"] ,
        df2["Summary"] ,
        df3["Summary"]
        ]).drop_duplicates().reset_index(drop=True)
    )

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, adf80132-92de-46cc-a964-d0df69a8d7b0)

In [39]:
names_to_filter = ['Metric', 'Metric_x100000000', 'Metric_x10000','Metric_x0']

df4 = pandas.concat([
        df1["Columns"].loc[df1["Columns"]['Column Name'].isin(names_to_filter)] ,
        df2["Columns"].loc[df2["Columns"]['Column Name'].isin(names_to_filter)] ,
        df3["Columns"].loc[df3["Columns"]['Column Name'].isin(names_to_filter)]
        ]
        ).drop_duplicates().reset_index(drop=True)
display(df4)
# Create chart

StatementMeta(, 7f97db67-7db4-4e1e-a84b-d2b25c596324, 47, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5e319e61-7678-4477-bea2-7757f3583991)

## Create Custom Semantic Model from Lakehouse

In [ ]:
#1. Generate list of ALL table names from lakehouse to add to Semantic Model
lakehouseTables:list = labs.lakehouse.get_lakehouse_tables(lakehouse=LakehouseName)["Table Name"]

#2. Remove any previously created semantic model of this name
for index, row in sempy.fabric.list_items().iterrows():
    if row["Type"] == "SemanticModel" and row["Display Name"] == SemanticModelName:
        sempy.fabric.delete_item(item_id=row["Id"],workspace=workspaceId)
        print(f"Deleted semantic model {row['Display Name']}")

#3 Create the semantic model
labs.directlake.generate_direct_lake_semantic_model(dataset=SemanticModelName,lakehouse_tables=lakehouseTables,workspace=workspaceName,lakehouse=lakehouseId,refresh=False,overwrite=True)

## Add Model Relationships

In [ ]:
with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    #1. Remove any existing relationships
    for r in tom.model.Relationships:
        tom.model.Relationships.Remove(r)

    #2. Creates correct relationships
    tom.add_relationship(from_table="Fact_A_Level"          , from_column="DateId"                , to_table="Dim_Date"                 , to_column="DateId"                , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level"          , from_column="AppAttributesKey"      , to_table="Dim_Attributes"           , to_column="AppAttributesKey"      , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level"          , from_column="CustomerAttributesKey" , to_table="Dim_CustomerAttributes"   , to_column="CustomerAttributesKey" , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level"          , from_column="DeviceAttributesKey"   , to_table="Dim_DeviceAttributes"     , to_column="DeviceAttributesKey"   , from_cardinality="many" , to_cardinality="one")

    tom.add_relationship(from_table="Fact_A_Level_Split_1"  , from_column="DateId"                , to_table="Dim_Date"                 , to_column="DateId"                , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_1"  , from_column="AppAttributesKey"      , to_table="Dim_Attributes"           , to_column="AppAttributesKey"      , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_1"  , from_column="CustomerAttributesKey" , to_table="Dim_CustomerAttributes"   , to_column="CustomerAttributesKey" , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_1"  , from_column="DeviceAttributesKey"   , to_table="Dim_DeviceAttributes"     , to_column="DeviceAttributesKey"   , from_cardinality="many" , to_cardinality="one")

    tom.add_relationship(from_table="Fact_A_Level_Split_2"  , from_column="DateId"                , to_table="Dim_Date"                 , to_column="DateId"                , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_2"  , from_column="AppAttributesKey"      , to_table="Dim_Attributes"           , to_column="AppAttributesKey"      , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_2"  , from_column="CustomerAttributesKey" , to_table="Dim_CustomerAttributes"   , to_column="CustomerAttributesKey" , from_cardinality="many" , to_cardinality="one")
    tom.add_relationship(from_table="Fact_A_Level_Split_2"  , from_column="DeviceAttributesKey"   , to_table="Dim_DeviceAttributes"     , to_column="DeviceAttributesKey"   , from_cardinality="many" , to_cardinality="one")

## Hide Columns in Fact Table

In [ ]:
tom = labs.tom.TOMWrapper(dataset=SemanticModelName, workspace=workspaceName, readonly=False)

i:int=0
for t in tom.model.Tables:
    if t.Name in ["Fact_A_Level","Fact_A_Level_Split_1","Fact_A_Level_Split_2"]:

        for c in t.Columns:
            c.IsHidden=True

        bim = json.dumps(tom.get_bim()["model"]["tables"][i],indent=4)
        print(bim)
    i=i+1

## Add Model Measures

In [ ]:
with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    #1. Remove any existing measures
    for t in tom.model.Tables:
        for m in t.Measures:
            tom.remove_object(m)
            print(f"Measure {m.Name} removed")

    tom.add_measure(table_name="Fact_A_Level",measure_name="Sum of Metric",   expression="SUM(Fact_A_Level[Metric])")
    tom.add_measure(table_name="Fact_A_Level",measure_name="Sum of Metric 1", expression="""
    
    	VAR a = SUM(Fact_A_Level_Split_1[Metric_x100000000])
		VAR b = SUM(Fact_A_Level_Split_1[Metric_x10000])
		VAR c = SUM(Fact_A_Level_Split_1[Metric_x0])
		
		RETURN  ((a*1000000) + (b*100) + (c/100.0000)) 
            """)

    tom.add_measure(table_name="Fact_A_Level",measure_name="Sum of Metric 2", expression="""
    
    	VAR a = SUM(Fact_A_Level_Split_2[Metric_x100000000])
		VAR b = SUM(Fact_A_Level_Split_2[Metric_x10000])
		VAR c = SUM(Fact_A_Level_Split_2[Metric_x0])
		
		RETURN ((a*1000000) + (b*100) + (c/100.00000)) 
            """)

## Reframe model to update changes

In [ ]:
reframeOK:bool=False
while not reframeOK:
    try:
        result:pandas.DataFrame = labs.refresh_semantic_model(dataset=SemanticModelName)
        reframeOK=True
    except:
        print('Error with reframe... trying again.')
        triggerMetadataRefresh()
        sleep(3)

print('Custom Semantic Model reframe OK')

## Run DMV to check column details

In [ ]:
def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

runDMV()

## Create function to run DAX query with a server timings trace

In [ ]:
import warnings
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

def filter_func(e):
    retVal:bool=True
    if e.EventSubclass.ToString() == "VertiPaqScanInternal":
        retVal=False      
    #     #if e.EventSubClass.ToString() == "VertiPaqScanInternal":
    #     retVal=False
    return retVal

# define events to trace and their corresponding columns
def runQueryWithTrace (expr:str,workspaceName:str,SemanticModelName:str,Result:Optional[bool]=True,Trace:Optional[bool]=True,DMV:Optional[bool]=True):
    event_schema = fabric.Trace.get_default_query_trace_schema()
    event_schema.update({"ExecutionMetrics":["EventClass","TextData"]})
    del event_schema['VertiPaqSEQueryBegin']
    del event_schema['VertiPaqSEQueryCacheMatch']
    del event_schema['DirectQueryBegin']

    warnings.filterwarnings("ignore")

    WorkspaceName = workspaceName
    SemanticModelName = SemanticModelName

    with fabric.create_trace_connection(SemanticModelName,WorkspaceName) as trace_connection:
        # create trace on server with specified events
        with trace_connection.create_trace(
            event_schema=event_schema, 
            name="Simple Query Trace",
            filter_predicate=filter_func,
            stop_event="QueryEnd"
            ) as trace:

            trace.start()

            df=sempy.fabric.evaluate_dax(
                dataset=SemanticModelName, 
                dax_string=expr)

            if Result:
                displayHTML(f"<H2>####### DAX QUERY RESULT #######</H2>")
                display(df)

            # Wait 5 seconds for trace data to arrive
            time.sleep(5)

            # stop Trace and collect logs
            final_trace_logs = trace.stop()

    if Trace:
        displayHTML(f"<H2>####### SERVER TIMINGS #######</H2>")
        display(final_trace_logs)
    
    if DMV:
        displayHTML(f"<H2>####### SHOW DMV RESULTS #######</H2>")
        runDMV()



## Run some DAX Queries

## First run to test time to page columns in

### Running SUM() over original column

In [ ]:
runQueryWithTrace("""

    EVALUATE
        {[Sum of Metric]}

""",workspaceName,SemanticModelName,Result=False,DMV=False)

### Running SUM() over new column in model still with original column

In [ ]:
runQueryWithTrace("""

    EVALUATE
        {[Sum of Metric 1]}

""",workspaceName,SemanticModelName)

In [ ]:
runQueryWithTrace("""

    EVALUATE
        {[Sum of Metric 2]}

""",workspaceName,SemanticModelName)

## Warm cache perf

Clear the DAX Query Cache and then run a query inside a Server Timings Trace

In [ ]:
labs.clear_cache(SemanticModelName)

runQueryWithTrace("""
    EVALUATE 
        SUMMARIZECOLUMNS(
            Dim_Date[DateId]

            , "SUM of Base" 	, [Sum of Metric]
            , "SUM of Metric 1" , [Sum of Metric 1]
            , "SUM of Metric 2" , [Sum of Metric 2]
            )
        ORDER BY [DateId]   

""",workspaceName,SemanticModelName,Result=False,DMV=False)

In [ ]:
df=sempy.fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string="""
    
    evaluate tabletraits()
    
    """)
display(df)

In [ ]:
runDMV()